In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# DataFrame with submits informotaion
data = pd.DataFrame(
    columns=['Rank', 
             'University',
             'Team Name', 
             'Problem Letter', 
             'Is Solved',
             'Attempts',
             'Problem Time', 
             'Total solved problems', 
             'Total Time', 
             'Year'
])

# DataFrame with problem information
problems_data = pd.DataFrame(
    columns=['Problem Letter', 'Problem Name', 'Year']
)


for year in range(2019, 2023):
    url = f'https://neerc.ifmo.ru/archive/{year}/standings.html'
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')

    problem_letters = []
    problem_names = []
    # Getting problems information
    for problem in soup.thead.find_all('th', {'class' : 'problem'}):
        problem_letters.append( problem.text )
        problem_names.append( problem['title'] )
        
        problems_data.loc[len(problems_data)] = [problem_letters[-1], problem_names[-1], year]
        
        
    # Getting submits infromation
    for participant in soup.tbody.find_all('tr'):
        all_columns = participant.find_all('td')
        
        try:
            rank = all_columns[0].text
            university = all_columns[1].text.split(':')[0].strip()
            team_name = ''.join(all_columns[1].text.split(':')[1:]).strip()
            total_solved_problems = all_columns[-3].text
            total_time = all_columns[-2].text
        except Exception:
            continue     
      
        if 'Yaroslavl' in university:
            university = 'Yaroslavl SU'

        while not university[-1].isalpha():
            university = university[:-1]

        for problem_letter, problem_info in zip( problem_letters, all_columns[2:-3] ):
            if problem_info.text == '.': # non solved problem without attempts
                data.loc[len(data)] = [
                    rank, 
                    university,
                    team_name, 
                    problem_letter, 
                    '0', 
                    '0', 
                    '0', 
                    total_solved_problems, 
                    total_time,
                    year
                ]
            elif problem_info.find('b') != None: # non solved problem wiht attempts
                data.loc[len(data)] = [
                    rank, 
                    university,
                    team_name, 
                    problem_letter, 
                    '0', 
                    problem_info.b.text, 
                    '0', 
                    total_solved_problems, 
                    total_time,
                    year
                ]
            else: #solved problem
                data.loc[len(data)] = [
                    rank, 
                    university,
                    team_name, 
                    problem_letter, 
                    '1', 
                    problem_info.i.find(text=True, recursive=False), 
                    problem_info.s.text, 
                    total_solved_problems, 
                    total_time,
                    year
                ]

In [3]:
problems_data.to_csv('problems.csv', index=False)
data.to_csv('contests.csv', index=False)